# Import Library


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Euclidean distance function

In [2]:
def compute_euclidean_distance_matrix(locations):
    """
    Compute Euclidean distance matrix for a list of locations.
    Args:
        locations: numpy array or list of shape (N, 2), with [x, y] coordinates for each node (depot + customers)
    Returns:
        distance_matrix: numpy array of shape (N, N) where (i, j) is the distance between node i and j
    """
    locations = np.array(locations)
    diff = locations[:, np.newaxis, :] - locations[np.newaxis, :, :]
    distance_matrix = np.sqrt(np.sum(diff**2, axis=-1))
    return distance_matrix


# Example usage:
# Suppose depot at (0,0), customer1 at (2,3), customer2 at (3,1), customer3 at (5,5)
locations = [
    [0, 0],  # Depot
    [2, 3],  # Customer 1
    [3, 1],  # Customer 2
    [5, 5],  # Customer 3
]
dist_matrix = compute_euclidean_distance_matrix(locations)
print(dist_matrix)

[[0.         3.60555128 3.16227766 7.07106781]
 [3.60555128 0.         2.23606798 3.60555128]
 [3.16227766 2.23606798 0.         4.47213595]
 [7.07106781 3.60555128 4.47213595 0.        ]]


# Import Data

In [3]:
# Load CSV files
df_customers = pd.read_excel(
    r"C:\rl-meta-test\src\Source\Solomon datasets\c101.xlsx", sheet_name="customer"
)
df_dataInfo = pd.read_excel(
    r"C:\rl-meta-test\src\Source\Solomon datasets\c101.xlsx", sheet_name="vehicle"
)

In [5]:
df_customers

,CUST NO.,XCOORD.,YCOORD.,DEMAND,READY TIME,DUE DATE,SERVICE TIME
0,0,40,50,0,0,1236,0
1,1,45,68,10,912,967,90
2,2,45,70,30,825,870,90
3,3,42,66,10,65,146,90
4,4,42,68,10,727,782,90
...,...,...,...,...,...,...,...
96,96,60,80,10,95,156,90
97,97,60,85,30,561,622,90
98,98,58,75,20,30,84,90
99,99,55,80,10,743,820,90


# Data Preparation

In [13]:
df_customers.iloc[:26, 1:3]

,XCOORD.,YCOORD.
0,40,50
1,45,68
2,45,70
3,42,66
4,42,68
5,42,65
6,40,69
7,40,66
8,38,68
9,38,70


In [15]:
customer = 10

# Extract coordinates (x, y) of customers (including depot) and convert to NumPy array
xycoord = df_customers.iloc[:customer, 1:3].to_numpy()
distance = compute_euclidean_distance_matrix(xycoord)  # Calculate distance
# distance = pd.read_csv(
#    r"C:\rl-meta-test\src\Source\solomon_data\solomon25_csv\distance_matrix\c101distanceMatrix.csv"
# ).to_numpy()


# Extract demand for each customer as NumPy array
demand = df_customers.iloc[:customer, 3].to_numpy()

# Extract 'ready time' (earliest time service can begin at each customer)
readyTime = df_customers.iloc[:customer, 4].to_numpy()

# Extract 'due date' (latest time service must begin at each customer)
dueDate = df_customers.iloc[:customer, 5].to_numpy()

# Extract service time required at each customer
serviceTime = df_customers.iloc[:customer, 6].to_numpy()

# Extract vehicle information (such as capacity, number of vehicles, etc.) from the first row of df_dataInfo
vehicle = df_dataInfo.iloc[0, :].to_numpy()

kwargs = {
    "distance": distance,
    "demand": demand,
    "readyTime": readyTime,
    "dueDate": dueDate,
    "serviceTime": serviceTime,
    "vehicle": vehicle,
}


# Differential Evolutional Algorithm Parameters

In [17]:
dimensions = len(distance) - 1 + vehicle[0]
maxiters = 10
n_pop = 10
bounds = np.array([[0, 1]] * dimensions)
Mutation_rate = np.array([0.9, 0.5])
Crossover_rate = np.array([0.5, 0.1])
island = 2
migration_rate = 0.5

# Differential Evolutional Algorithm


In [ ]:
def differential_evolution(
    objective_func,
    bounds,
    population_size=n_pop,
    max_generations=maxiters,
    Mutation_rate=Mutation_rate,
    Crossover_rate=Crossover_rate,
    island=island,
    migration_rate=migration_rate,
    **kwargs,
):
    global_solution = np.array([])
    
    for num_island in range(island):
        if num_island == 0:
            # Initialize population
            population = np.random.uniform(
                bounds[:, 0], bounds[:, 1], (population_size, len(bounds))
            )
        else:
            # Initialize population
            population = np.random.uniform(
                bounds[:, 0], bounds[:, 1], (population_size, len(bounds))
            )
            migration_cost = np.array([])
            for obj in range(len(population)):
                migration_cost = np.insert(
                    migration_cost,
                    len(migration_cost),
                    objective_func(population[obj], **kwargs),
                )

            idx_good_migration = migration_cost.argsort()
            idx_bad_migration = np.flip(idx_good_migration)[
                : int(population_size * migration_rate)
            ]

            population[idx_bad_migration] = migration_population

        Upperbound_Mutation = Mutation_rate[1]
        Lowerbound_Mutation = Mutation_rate[0]
        Upperbound_Crossover_rate = Crossover_rate[1]
        Lowerbound_Crossover_rate = Crossover_rate[0]
        F = Mutation_rate[0]
        CR = Crossover_rate[0]

        for generation in range(max_generations):
            # print(f'Iteration {generation}')
            current_cost = np.array([])
            F += (Upperbound_Mutation - Lowerbound_Mutation) / max_generations
            CR += (
                Upperbound_Crossover_rate - Lowerbound_Crossover_rate
            ) / max_generations
            for i in range(population_size):
                # Mutation
                indices = [idx for idx in range(population_size) if idx != i]
                a, b, c = population[np.random.choice(indices, 3, replace=False)]
                mutant = population[i] + F * (b - c)

                # Crossover
                crossover_prob = np.random.rand(len(bounds))
                trial = np.where(crossover_prob < CR, mutant, population[i])

                # Selection
                fitness_trial = objective_func(trial, **kwargs)
                fitness_current = objective_func(population[i], **kwargs)

                if fitness_trial < fitness_current:
                    population[i] = trial
                    current_cost = np.insert(
                        current_cost, len(current_cost), fitness_trial
                    )
                else:
                    current_cost = np.insert(
                        current_cost, len(current_cost), fitness_current
                    )

            idx_best_migration = current_cost.argsort()[
                : int(population_size * migration_rate)
            ]
            migration_population = population[idx_best_migration]
            

            best_index_plot = current_cost[np.argmin(current_cost)]
            global_solution = np.insert(
                global_solution, len(global_solution), best_index_plot
            )

        # Find the best solution
        best_index = np.argmin(
            [objective_func(individual, **kwargs) for individual in population]
        )
        best_solution = population[best_index]

    return best_solution, global_solution

# Decoding

In [19]:
def objective_func(x, **kwargs):
    vehicle = kwargs["vehicle"]
    seq = x[: -vehicle[0]].argsort() + 1
    sort = x[-vehicle[0] :].argsort()
    j = f_per_particle(
        seq, sort, **kwargs
    )  # [f_per_particle1(x[i], sort[i], i, **kwargs) if i <= 2 else f_per_particle(x[i], sort[i], **kwargs) for i in range(n_particles)]        # Calculate each particle.
    return np.array(j)


def f_per_particle(m, s, **kwargs):
    X = m  # Sequence
    V = s  # Vehicle
    obj_val = preserving_strategy(X, V, **kwargs)  # Call Preserving strategy.
    return obj_val


def preserving_strategy(X, V, **kwargs):
    # --- Unpack input data from keyword arguments ---
    dist = kwargs["distance"]  # Distance/time matrix between all nodes
    weight = kwargs["demand"]  # Demand (weight) for each customer node
    ready = kwargs["readyTime"]  # Ready time (earliest service time) for each node
    due = kwargs["dueDate"]  # Due time (latest service time) for each node
    service = kwargs["serviceTime"]  # Service time at each node
    vehicle = kwargs[
        "vehicle"
    ]  # Vehicle info: [number of vehicles, capacity per vehicle]
    # Get per-vehicle capacities (by indexing with V)
    pre_w_cap = np.array([vehicle[1]] * vehicle[0])
    w_cap = pre_w_cap[V]

    # -- Initialization --
    sequence = X  # Route sequence (includes depot at start & end)
    n_cust = len(sequence) - 2  # Number of customers (not counting depot nodes)
    n_veh = vehicle[0] - 1  # Number of vehicles - 1 (for indexing)
    i, k = 0, 0  # i: current position in sequence, k: vehicle index
    total_distance = 0  # Store total traveled distance (with penalty if any)

    # -- Main loop over each vehicle route --
    while k <= n_veh and i <= n_cust:
        # Initialize per-route accumulators
        route_dist, route_time, weight_load, penaltyCost = 0, 0, 0, 0

        if k > 0:
            i += 1  # Move to the next start customer for the next vehicle
        # Start route: depot to first customer
        route_dist += dist[0][sequence[i]]  # Distance depot -> first customer
        route_time += (
            service[0] + dist[0][sequence[i]]
        )  # Service + travel time to first customer
        weight_load += weight[sequence[i]]  # Initial cargo: first customer demand

        if route_time < ready[sequence[i]]:
            route_time = ready[sequence[i]]  # Wait if vehicle arrives before ready time

        if route_time > due[sequence[i]] or weight_load > w_cap[k]:
            penaltyCost += 1e11  # Penalty: arrived after due time (infeasible)
            break

        # --- Continue visiting customers along this route ---
        while i <= n_cust:
            route_dist += dist[sequence[i]][sequence[i + 1]]  # Add next leg distance
            route_time += (
                service[sequence[i]] + dist[sequence[i]][sequence[i + 1]]
            )  # Add service + travel time
            weight_load += weight[sequence[i + 1]]  # Add new customer demand

            if route_time < ready[sequence[i + 1]]:
                route_time = ready[sequence[i + 1]]  # Wait if arrive early at next node

            # If time window or capacity violated, backtrack and finish route
            if route_time > due[sequence[i + 1]] or weight_load > w_cap[k]:
                route_dist -= dist[sequence[i]][sequence[i + 1]]
                route_time -= service[sequence[i]] + dist[sequence[i]][sequence[i + 1]]
                weight_load -= weight[sequence[i + 1]]
                break
            i += 1

        # --- Finish by returning to depot ---
        route_dist += dist[sequence[i]][0]  # Add distance to depot
        route_time += (
            service[sequence[i]] + dist[sequence[i]][0]
        )  # Add service at last node + travel to depot
        if route_time > due[0]:
            penaltyCost += 1e11  # Penalty: returned to depot too late
        # Accumulate this route's total (distance + penalty if any)
        total_distance += route_dist + penaltyCost
        # print(f'Total weight {weight_load} km.')
        # print(f'Traveling time {route_time} mins.')
        # print(f'Total traveling distance {route_dist} km.')
        k += 1  # Next vehicle

    return (
        total_distance  # Return overall objective (distance with penalty if violated)
    )


# Checking Solution

In [20]:
correct_solution = np.array([0, 1, 2, 3, 4])
correct_solution[:-1]

array([0, 1, 2, 3])

In [42]:
def checking_solution(X, V, **kwargs):
    # --- Unpack input data from keyword arguments ---
    dist = kwargs["distance"]  # Distance/time matrix between all nodes
    weight = kwargs["demand"]  # Demand (weight) for each customer node
    ready = kwargs["readyTime"]  # Ready time (earliest service time) for each node
    due = kwargs["dueDate"]  # Due time (latest service time) for each node
    service = kwargs["serviceTime"]  # Service time at each node
    vehicle = kwargs[
        "vehicle"
    ]  # Vehicle info: [number of vehicles, capacity per vehicle]
    # Get per-vehicle capacities (by indexing with V)
    pre_w_cap = np.array([vehicle[1]] * vehicle[0])
    w_cap = pre_w_cap[V]

    # -- Initialization --
    sequence = X  # Route sequence (includes depot at start & end)
    n_cust = len(sequence) - 2  # Number of customers (not counting depot nodes)
    n_veh = vehicle[0] - 1  # Number of vehicles - 1 (for indexing)
    i, k = 0, 0  # i: current position in sequence, k: vehicle index
    total_distance = 0  # Store total traveled distance (with penalty if any)

    correct_solution = np.array([])

    # -- Main loop over each vehicle route --
    while k <= n_veh and i <= n_cust:
        # Initialize per-route accumulators
        route_dist, route_time, weight_load, penaltyCost = 0, 0, 0, 0

        if k > 0:
            i += 1  # Move to the next start customer for the next vehicle
        print(i)
        # Start route: depot to first customer
        route_dist += dist[0][sequence[i]]  # Distance depot -> first customer
        route_time += service[0] + dist[0][sequence[i]]
        
        correct_solution = np.insert(correct_solution, len(correct_solution), 0)
        

        # Service + travel time to first customer
        weight_load += weight[sequence[i]]  # Initial cargo: first customer demand

        if route_time < ready[sequence[i]]:
            route_time = ready[sequence[i]]  # Wait if vehicle arrives before ready time

        if route_time > due[sequence[i]] or weight_load > w_cap[k]:
            penaltyCost += 1e11  # Penalty: arrived after due time (infeasible)
            break

        # --- Continue visiting customers along this route ---
        while i <= n_cust:
            route_dist += dist[sequence[i]][sequence[i + 1]]  # Add next leg distance
            route_time += service[sequence[i]] + dist[sequence[i]][sequence[i + 1]]

            correct_solution = np.insert(
                correct_solution, len(correct_solution), sequence[i]
            )

            # Add service + travel time
            weight_load += weight[sequence[i + 1]]  # Add new customer demand

            if route_time < ready[sequence[i + 1]]:
                route_time = ready[sequence[i + 1]]  # Wait if arrive early at next node

            # If time window or capacity violated, backtrack and finish route
            if route_time > due[sequence[i + 1]] or weight_load > w_cap[k]:
                route_dist -= dist[sequence[i]][sequence[i + 1]]
                route_time -= service[sequence[i]] + dist[sequence[i]][sequence[i + 1]]
                weight_load -= weight[sequence[i + 1]]

                correct_solution = correct_solution[:-1]
                break
            i += 1

        # --- Finish by returning to depot ---
        route_dist += dist[sequence[i]][0]  # Add distance to depot
        route_time += (
            service[sequence[i]] + dist[sequence[i]][0]
        )  # Add service at last node + travel to depot
    

        if route_time > due[0]:
            penaltyCost += 1e11  # Penalty: returned to depot too late
        # Accumulate this route's total (distance + penalty if any)
        total_distance += route_dist + penaltyCost

        k += 1  # Next vehicle

    return correct_solution

# Finding Solution


In [23]:
start_algorithm = time.time()
best_solution, global_solution_plot = differential_evolution(
    objective_func,
    bounds=bounds,
    population_size=n_pop,
    max_generations=maxiters,
    Mutation_rate=Mutation_rate,
    Crossover_rate=Crossover_rate,
    island=island,
    migration_rate=migration_rate,
    **kwargs,
)
End_algorithm = time.time()
run_algorithm = End_algorithm - start_algorithm

In [32]:
seq = best_solution[: -vehicle[0]].argsort() + 1
sort = best_solution[-vehicle[0] :].argsort()
print(f'Sequence : {seq}')
print(f'Vehicle  : {sort}')

Sequence : [5 3 4 2 1 8 9 6 7]
Vehicle  : [ 2 22 15  4 10 12 11 17 16 20 23  7  5 19  1  8  3  6 13  0 14 21 18  9
 24]


In [30]:
preserving_strategy(seq, sort, **kwargs)

np.float64(115.76667717192957)

In [43]:
checking_solution(seq, sort, **kwargs)

0
5
8


array([0., 5., 3., 4., 2., 0., 8., 9., 0.])

# Plotting

In [ ]:
Replication = [i for i in range(len(global_solution_plot))]
y1 = global_solution_plot.tolist()
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.plot(Replication, y1, marker=".", label="Differential Evoluation Algorithm")
ax.set(
    xlabel="iteration",
    ylabel="Total Distance (Km.)",
    title="Differential Evoluation Algorithm Replication",
)
ax.grid()
ax.legend()
plt.show()

In [ ]:
y1[-1]